# Complex Net
blabla

In [1]:
# A bit of setup
from __future__ import print_function
import os,sys,inspect
import torch
import torch.optim as optim
import numpy as np
import time
import random
from scipy import fft
from torch import nn
from resampy import resample
from sklearn.metrics import average_precision_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from intervaltree import Interval, IntervalTree


from music.resample import resample_musicnet
from c2nn.utils import SignalDataset_music
from c2nn.model import TransformerModel,TransformerGenerationModel
from c2nn.train import train_transformer,train_model

train len 0
train len 0


# Data process


In [8]:
#resample
resample_musicnet("music/musicnet.npz", "music/musicnet_11khz.npz", 44100, 11000)

.. resampling music/musicnet.npz (44100Hz) into music/musicnet_11khz.npz (11000Hz)
.. sampling with ratio 0.2494331065759637
.. aggregating 1788 (0 / 330)


In [3]:
#parse_file, get .npy files

fs = 11000            # samples/second
window_size = 4096    # fourier window size
d = 2048              # number of features
m = 128               # (USED BY DCN) number of distinct notes
stride = 512         # samples between windows
stride_test = 128            # stride in test set
k = 64            # number of window (time step) per piece
k_test = 64
data = np.load(open('music/musicnet_11khz.npz','rb'), encoding='latin1')

test_data = ['2303','2382','1819']
train_data = [f for f in data.files if f not in test_data]
index = 0

In [4]:
"""
create train data set
"""
print("len(train_data)",len(train_data))
for i in range(len(train_data)):
    print(i)
    X,Y = data[train_data[i]]
    for p in range(int((len(X)-window_size)/stride/k)):
        Xtrain = np.empty([k,d,2])
        Ytrain = np.zeros([k,m])
        for j in range(k):
            s = j*stride+p*k*stride# start from one second to give us some wiggle room for larger segments
            X_fft = fft(X[s:s+window_size])
            Xtrain[j, :, 0] = X_fft[0:d].real
            Xtrain[j, :, 1] = X_fft[0:d].imag
            # label stuff that's on in the center of the window
            for label in Y[s+d/2]:
                if (label.data[1]) >= m:
                    continue
                else:
                    Ytrain[j,label.data[1]] = 1
        Xtrain = Xtrain.reshape(k, d*2, order='F')
        np.save("music/music_train_x_64_{}.npy".format(index), Xtrain)
        np.save("music/music_train_y_64_{}.npy".format(index), Ytrain)
        index = index + 1
print("train data saved")

len(train_data) 327
0
1


KeyboardInterrupt: 

In [5]:
""" 
create the test set
"""
index = 0
for i in range(len(test_data)):
    print(i)
    X,Y = data[test_data[i]]
    for p in range(int((len(X)-window_size)/stride_test/k_test)):
        Xtest = np.empty([k_test,d,2])
        Ytest = np.zeros([k_test,m])
        for j in range(k_test):
            s = j*stride_test+p*k_test*stride_test
            # start from one second to give us some wiggle room for larger segments
            X_fft = fft(X[s:s+window_size])
            Xtest[j, :, 0] = X_fft[0:d].real
            Xtest[j, :, 1] = X_fft[0:d].imag           
            # label stuff that's on in the center of the window
            for label in Y[s+d/2]:
                if (label.data[1]) >= m:
                    continue
                else:
                    Ytest[j,label.data[1]] = 1
        Xtest = Xtest.reshape(k_test, d*2, order='F')
        np.save("music/music_test_x_64_{}.npy".format(index), Xtest)
        np.save("music/music_test_y_64_{}.npy".format(index), Ytest)
        index = index + 1
print("finished")

0
1


KeyboardInterrupt: 

# Train



In [3]:
args = {
    'seed': 1111,
    'attn_dropout': 0.0, 
    'attn_mask' : False,
    'batch_size' : 16,
    'clip' : 0.35,
    'data' : 'music',
    'embed_dim' : 320,
    'hidden_size' : 2048,
    'lr' : 1e-4,
    'modal_lengths' : [2048, 2048],
    'model' : 'Transformer',
    'nlevels' : 6,
    'num_epochs' : 2000,
    'num_heads' : 8,
    'optim' : 'Adam',
    'out_dropout' : 0.5,
    'output_dim' : 128,
    'path' : 'music/',
    'relu_dropout' : 0.1,
    'res_dropout' : 0.1,
    'time_step' : 64
}

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])
np.random.seed(args['seed'])
random.seed(args['seed'])
torch.backends.cudnn.deterministic = True


In [ ]:
"""
Data Loading
"""
torch.set_default_tensor_type('torch.FloatTensor')
print("Start loading the data....")
start_time = time.time() 

training_set = SignalDataset_music(args['path'], args['time_step'], train=True)
test_set = SignalDataset_music(args['path'], args['time_step'], train=False)

print("Finish loading the data....")
train_loader = torch.utils.data.DataLoader(training_set, batch_size=args['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=args['batch_size'], shuffle=True)

"""
Train!!!
"""
train_transformer(args,training_set,train_loader,test_set,test_loader)